## RAG from scratch

This section aims to guide you through the process of building a basic RAG from scratch.

# Basic RAG

Retrieval-augmented generation (RAG) is an AI framework that synergizes the capabilities of LLMs and information retrieval systems. It’s useful to answer questions or generate content leveraging external knowledge.

There are two main steps in RAG:
1. retrieval: retrieve relevant information from a knowledge base with text embeddings stored in a vector store;
2. generation: insert the relevant information to the prompt for the LLM to generate information.

In this guide, we will walk through a very basic example of RAG with four implementations:

- RAG from scratch with Llama 3.2 (using free Groq API),  and Faiss
- RAG with Llama 3.2 and LangChain


### Setup and Installation

Install the required libraries

In [1]:
!pip install numpy==1.26.4 faiss-cpu==1.8.0 openai==1.14.3 sentence-transformers pandas==1.5.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.6.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 re

Download the PubMedQA Labeled for demonstration purposes.

In [2]:
!wget https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json -O pqa_labelled.json

--2024-10-17 04:49:50--  https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2584787 (2.5M) [text/plain]
Saving to: ‘pqa_labelled.json’

pqa_labelled.json   100%[===================>]   2.46M  --.-KB/s    in 0.08s   

2024-10-17 04:49:50 (29.9 MB/s) - ‘pqa_labelled.json’ saved [2584787/2584787]



### Import Libraries

In [3]:
import numpy as np
import faiss
import json
import pandas as pd

from sentence_transformers import SentenceTransformer
from openai import OpenAI

client = OpenAI(
    api_key="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",
    base_url='https://api.groq.com/openai/v1',
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Load and view the dataset

In [4]:
with open('pqa_labelled.json', 'r') as f:
    data = json.load(f)

transformed_data = {
    "questions": [],
    "contexts": [],
    "answers": []
}

for item in data.values():
    transformed_data['questions'].append(item['QUESTION'])
    transformed_data['contexts'].append(item['CONTEXTS'])
    transformed_data['answers'].append(item['LONG_ANSWER'])

df = pd.DataFrame(transformed_data)
df.head()

,questions,contexts,answers
0,Do mitochondria play a role in remodelling lac...,[Programmed cell death (PCD) is the regulated ...,Results depicted mitochondrial dynamics in viv...
1,Landolt C and snellen e acuity: differences in...,[Assessment of visual acuity depends on the op...,"Using the charts described, there was only a s..."
2,"Syncope during bathing in infants, a pediatric...",[Apparent life-threatening events in infants a...,"""Aquagenic maladies"" could be a pediatric form..."
3,Are the long-term results of the transanal pul...,[The transanal endorectal pull-through (TERPT)...,Our long-term study showed significantly bette...
4,Can tailored interventions increase mammograph...,[Telephone counseling and tailored print commu...,The effects of the intervention were most pron...


## Split document into chunks

In a RAG system, it is crucial to split the document into smaller chunks so that it’s more effective to identify and retrieve the most relevant information in the retrieval process later. In this example, we simply split our text by character, combine 2048 characters into each chunk.

In [5]:
chunk_size = 2048
chunks = []

for text in df['contexts']:
    text = "\n".join(text)
    chunks += [text[i:i + chunk_size] for i in range(0, len(text), chunk_size) if len(text[i:i + chunk_size]) > 0]

print(f"Total chunks: {len(chunks)}")

chunks = chunks[:200]
print(f"For testing, we will use only {len(chunks)} chunks")

Total chunks: 1033
For testing, we will use only 200 chunks


In [6]:
def get_text_embedding(sentences):
    model = SentenceTransformer('BAAI/bge-large-en-v1.5')
    embeddings = model.encode(sentences)
    return embeddings

In [7]:
text_embeddings = get_text_embedding(chunks)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [8]:
text_embeddings.shape

(200, 1024)

In [9]:
text_embeddings

array([[-0.0206785 , -0.02068686, -0.00386645, ..., -0.02460489,
        -0.01016388, -0.01389027],
       [ 0.02316779,  0.00813706,  0.01444898, ...,  0.00461098,
        -0.0086088 , -0.02615978],
       [-0.01896976,  0.0346964 ,  0.01957824, ..., -0.03605655,
         0.01847291, -0.03471109],
       ...,
       [ 0.0179466 ,  0.00674204, -0.01692699, ..., -0.00258184,
        -0.05088448, -0.01757879],
       [ 0.01564134,  0.02095525,  0.02021168, ..., -0.02712116,
         0.00242136, -0.03162438],
       [ 0.00582403, -0.00561286,  0.00256976, ..., -0.00079752,
        -0.00475964,  0.02209357]], dtype=float32)

### Load into a vector database
Once we get the text embeddings, a common practice is to store them in a vector database for efficient processing and retrieval. There are several vector database to choose from. In our simple example, we are using an open-source vector database Faiss, which allows for efficient similarity search.  

With Faiss, we instantiate an instance of the Index class, which defines the indexing structure of the vector database. We then add the text embeddings to this indexing structure.


In [10]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

### Create embeddings for a question
Whenever users ask a question, we also need to create embeddings for this question using the same embedding models as before.


In [11]:
question = "Explain what is Amblyopia?"
question_embeddings = get_text_embedding([question])
question_embeddings.shape

(1, 1024)

In [12]:
question_embeddings

array([[ 0.02755284, -0.02487543, -0.04647457, ...,  0.00898422,
        -0.00786411, -0.02185954]], dtype=float32)

### Retrieve similar chunks from the vector database
We can perform a search on the vector database with `index.search`, which takes two arguments: the first is the vector of the question embeddings, and the second is the number of similar vectors to retrieve. This function returns the distances and the indices of the most similar vectors to the question vector in the vector database. Then based on the returned indices, we can retrieve the actual relevant text chunks that correspond to those indices.


In [13]:
D, I = index.search(question_embeddings, k=3)
print(I)

[[ 26   1 106]]


In [14]:
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
print(retrieved_chunk)

['The records of 465 patients with an established diagnosis of age related macular degeneration who had attended a specialist macular clinic between 1990 and 1998 were scrutinised. A full clinical examination and standardised refraction had been carried out in 189 of these cases on a minimum of two occasions. Cases were looked for where an improvement of one or more lines of either distance or near acuity was recorded in the eye unaffected by macular disease. In each one of these cases the improvement in visual acuity could not be attributed to treatment of other existing pathology.\n12 such cases were detected. In nine of these the eye showing improvement of acuity had a history of amblyopia. The mean improvement in distance and near acuity in amblyopic eyes by 12 months was 3.3 and 1.9 lines logMAR respectively. The improvement in acuity generally occurred between 1 and 12 months from baseline and remained stable over the period of follow up.', 'Assessment of visual acuity depends on

### Combine context and question in a prompt and generate response

Finally, we can offer the retrieved text chunks as the context information within the prompt. Here is a prompt template where we can include both the retrieved text and user question in the prompt.



In [ ]:
def prompt_template(question, retrieved_chunk):
    prompt = f"""
    Answer the following question based only on the provided context.

    <context>
    {retrieved_chunk}
    </context>

    Question: {question}
    """
    return prompt

In [ ]:
def run_llm(user_message, model="llama-3.2-3b-preview"):
    system_message = "You are a helpful assistant. You are given a question and a context. You need to answer the question based on the context."
    messages = [{"role": "system", "content": system_message}]
    messages += [{"role": "user", "content": user_message}]
    completion = client.chat.completions.create(
        model=model,
        messages=messages,
    )
    return completion.choices[0].message.content

In [ ]:
prompt = prompt_template(question, retrieved_chunk)

run_llm(prompt)

'Based on the provided context, Amblyopia is a condition that can cause visual impairments in the eyes of individuals, particularly those with a history of strabismus (crossed eyes). In the context of the study, Amblyopic eyes showed improvement in visual acuity, with a mean improvement of 3.3 lines in distance acuity and 1.9 lines in near acuity over 12 months. It is also mentioned that in cases where the eye showing improvement was also amblyopic, the improvement occurred between 1 and 12 months from baseline and remained stable over the study period.'

### Test the dataset

In [ ]:
data_id = 1

question, answers = df['questions'][data_id], df['answers'][data_id]

question_embeddings = get_text_embedding([question])
D, I = index.search(question_embeddings, k=3)
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]
prompt = prompt_template(question, retrieved_chunk)

response = run_llm(prompt)
print("Question:", question)
print("RAG Response:", response)
print("Ground Truth:", answers)

Question: Landolt C and snellen e acuity: differences in strabismus amblyopia?
RAG Response: Based on the given context, the text mentions the following points regarding the differences in Landolt C and Snellen E acuity in the group of patients with strabismus amblyopia:

- The mean decimal values for Landolt C acuity (LR) and Snellen E acuity (SE) in this group were 0.14 and 0.16, respectively.
- The mean difference between LR and SE was 0.55 lines in this group.

So, in strabismus amblyopia, the Landolt C acuity was slightly worse compared to the Snellen E acuity, resulting in a 0.55 line difference between the two.
Ground Truth: Using the charts described, there was only a slight overestimation of visual acuity by the Snellen E compared to the Landolt C, even in strabismus amblyopia. Small differences in the lower visual acuity range have to be considered.


## LangChain

In [ ]:
!pip install langchain==0.1.13 langchain-community==0.0.29 langchain-openai==0.1.1 langchain-huggingface==0.0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found exist

In [ ]:
from langchain_core.documents import Document
from langchain_community.chat_models.openai import ChatOpenAI
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain


In [ ]:
# Load data
docs = [Document(page_content="\n".join(doc)) for doc in df['contexts']]
docs = docs[:200]


In [ ]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)


In [ ]:
# Define the embedding model
embeddings = SentenceTransformerEmbeddings(model_name="BAAI/bge-large-en-v1.5")
# Create the vector store
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()


In [ ]:
# Define LLM
model = ChatOpenAI(
    model_name='llama-3.2-3b-preview',
    openai_api_key="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",
    openai_api_base="https://api.groq.com/openai/v1"
)



In [ ]:
# Define prompt template
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
""")

In [ ]:
# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "Explain what is Amblyopia?"})
print(response["answer"])

In [ ]:
data_id = 1

question, answers = df['questions'][data_id], df['answers'][data_id]


response = retrieval_chain.invoke({"input": question})
print("Question:", question)
print("RAG Response:", response["answer"])
print("Ground Truth:", answers)